# Estimating School Location

This notebook illustrates how to re-estimate ActivitySim's auto ownership model.  The steps in the process are:
  - Run ActivitySim in estimation mode to read household travel survey files, run the ActivitySim submodels to write estimation data bundles (EDB) that contains the model utility specifications, coefficients, chooser data, and alternatives data for each submodel.
  - Read and transform the relevant EDB into the format required by the model estimation package [larch](https://larch.newman.me) and then re-estimate the model coefficients.  No changes to the model specification will be made.
  - Update the ActivitySim model coefficients and re-run the model in simulation mode.
  
The basic estimation workflow is shown below and explained in the next steps.

![estimation workflow](https://github.com/RSGInc/activitysim/raw/develop/docs/images/estimation_example.jpg)

# Load libraries

In [1]:
import larch  # !conda install larch #for estimation
import larch.util.activitysim
import pandas as pd
import numpy as np
import yaml 
import larch.util.excel
import os

# Required Inputs

In addition to a working ActivitySim model setup, estimation mode requires an ActivitySim format household travel survey.  An ActivitySim format household travel survey is very similar to ActivitySim's simulation model tables:

 - households
 - persons
 - tours
 - joint_tour_participants
 - trips (not yet implemented)

Examples of the ActivitySim format household travel survey are included in the [example_estimation data folders](https://github.com/RSGInc/activitysim/tree/develop/activitysim/examples/example_estimation).  The user is responsible for formatting their household travel survey into the appropriate format.  

After creating an ActivitySim format household travel survey, the `scripts/infer.py` script is run to append additional calculated fields.  An example of an additional calculated field is the `household:joint_tour_frequency`, which is calculated based on the `tours` and `joint_tour_participants` tables.  

The input survey files are below.

### Survey households

In [56]:
pd.read_csv("../data_sf/survey_data/override_households.csv")

,household_id,TAZ,income,hhsize,HHT,auto_ownership,num_workers,joint_tour_frequency
0,2223759,16,144100,2,1,0,2,1_Main
1,990869,134,48000,2,1,2,2,0_tours
2,125886,113,25900,1,4,1,1,0_tours
3,727893,8,26100,2,1,0,1,0_tours
4,2741769,150,121600,4,1,2,1,0_tours
...,...,...,...,...,...,...,...,...
1995,663493,110,19180,1,6,1,1,0_tours
1996,569375,20,7400,1,6,1,0,0_tours
1997,1445193,17,75000,1,4,0,1,0_tours
1998,2833455,69,0,1,0,0,0,0_tours


### Survey persons

In [57]:
pd.read_csv("../data_sf/survey_data/override_persons.csv")

,person_id,household_id,age,PNUM,sex,pemploy,pstudent,ptype,school_taz,workplace_taz,free_parking_at_work,cdap_activity,mandatory_tour_frequency,_escort,_shopping,_othmaint,_othdiscr,_eatout,_social,non_mandatory_tour_frequency
0,166,166,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,0,0,1,0,4
1,197,197,46,1,2,3,3,4,-1,-1,False,N,NaN,0,1,0,0,0,0,16
2,268,268,46,1,1,3,3,4,-1,-1,False,N,NaN,0,0,1,1,0,0,9
3,375,375,54,1,2,3,3,4,-1,-1,False,N,NaN,0,0,1,0,0,0,8
4,387,387,44,1,2,3,3,4,-1,-1,False,N,NaN,1,0,0,1,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4401,7554799,2863464,93,1,2,3,3,5,-1,-1,False,N,NaN,0,0,0,1,0,0,1
4402,7554818,2863483,68,1,1,3,3,5,-1,-1,False,N,NaN,0,0,1,1,0,0,9
4403,7555141,2863806,93,1,2,3,3,5,-1,-1,False,N,NaN,0,2,0,1,0,0,17
4404,7555853,2864518,71,1,1,3,3,5,-1,-1,False,N,NaN,0,0,0,0,0,1,2


### Survey tours

In [58]:
pd.read_csv("../data_sf/survey_data/override_tours.csv")

,tour_id,survey_tour_id,person_id,household_id,tour_type,tour_category,destination,origin,start,end,tour_mode,survey_parent_tour_id,parent_tour_id,composition,tdd,atwork_subtour_frequency
0,25820,258200,629,629,school,mandatory,133.0,131.0,12.0,15.0,WALK,NaN,NaN,NaN,115,NaN
1,52265,522650,1274,1274,school,mandatory,188.0,166.0,9.0,15.0,WALK_LOC,NaN,NaN,NaN,76,NaN
2,1117937,11179370,27266,27266,school,mandatory,133.0,9.0,17.0,18.0,WALK_HVY,NaN,NaN,NaN,163,NaN
3,1148523,11485230,28012,28012,school,mandatory,12.0,10.0,17.0,22.0,WALK_LRF,NaN,NaN,NaN,167,NaN
4,1208547,12085470,29476,29476,school,mandatory,13.0,16.0,8.0,15.0,WALK_LOC,NaN,NaN,NaN,61,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5768,302942627,3029426270,7388844,2750003,maint,atwork,5.0,7.0,14.0,14.0,WALK,3.029426e+09,302942643.0,NaN,135,NaN
5769,305120465,3051204650,7441962,2758909,maint,atwork,110.0,2.0,12.0,13.0,SHARED2FREE,3.051205e+09,305120481.0,NaN,113,NaN
5770,308000655,3080006550,7512211,2820876,eat,atwork,14.0,1.0,12.0,13.0,WALK,3.080007e+09,308000690.0,NaN,113,NaN
5771,308073840,3080738400,7513996,2822661,eat,atwork,69.0,107.0,8.0,16.0,SHARED3FREE,3.080739e+09,308073875.0,NaN,62,NaN


### Survey joint tour participants

In [59]:
pd.read_csv("../data_sf/survey_data/survey_joint_tour_participants.csv")

,participant_id,tour_id,household_id,person_id,participant_num
0,22095828301,220958283,2223759,5389226,1
1,22095828302,220958283,2223759,5389227,2
2,14429508701,144295087,1606646,3519392,1
3,14429508702,144295087,1606646,3519393,2
4,28367651801,283676518,2628704,6918939,1
...,...,...,...,...,...
226,16297928102,162979281,1769918,3975105,2
227,16297928103,162979281,1769918,3975106,3
228,16297928104,162979281,1769918,3975107,4
229,26353054902,263530549,2519358,6427575,1


# Example Setup if Needed

To avoid duplication of inputs, especially model settings and expressions, the `example_estimation` depends on the `example`.  The following commands create an example setup for use.  The location of these example setups (i.e. the folders) are important because the paths are referenced in this notebook.  The commands below download the skims.omx for the SF county example from the [activitysim resources repository](https://github.com/RSGInc/activitysim_resources).

In [37]:
!activitysim create -e example_estimation_sf -d test

copying configs ...
copying configs ...
copying data_sf ...
downloading skims.omx ...
copying output ...
copied! new project files are in C:\projects\development\activitysim_rsg\activitysim\examples\example_estimation\notebooks\test\test
the copied example can be run with

  activitysim run -w test


# Run the Estimation Example

The next step is to run the model with an `estimation.yaml` settings file with the following settings in order to output the EDB for all submodels:

```
enable=True

bundles:
  - school_location
  - workplace_location
  - auto_ownership
  - free_parking
  - cdap
  - mandatory_tour_frequency
  - mandatory_tour_scheduling
  - joint_tour_frequency
  - joint_tour_composition
  - joint_tour_participation
  - joint_tour_destination
  - joint_tour_scheduling
  - non_mandatory_tour_frequency
  - non_mandatory_tour_destination
  - non_mandatory_tour_scheduling
  - tour_mode_choice
  - atwork_subtour_frequency
  - atwork_subtour_destination
  - atwork_subtour_scheduling
  - atwork_subtour_mode_choice

survey_tables:
  households:
    file_name: survey_data/override_households.csv
    index_col: household_id
  persons:
    file_name:  survey_data/override_persons.csv
    index_col: person_id
  tours:
    file_name:  survey_data/override_tours.csv
  joint_tour_participants:
    file_name:  survey_data/override_joint_tour_participants.csv
```

This enables the estimation mode functionality, identifies which models to run and their output estimation data bundles (EDBs), and the input survey tables, which include the override settings for each model choice.  

With this setup, the model will output an EBD with the following tables for this submodel:
  - model settings - school_location_model_settings.yaml
  - coefficients - school_location_coefficients.csv
  - utilities specification - school_location_SPEC.csv
  - land use data - school_location_landuse.csv
  - size terms - school_location_size_terms.csv
  - alternatives values - school_location_alternatives_combined.csv
  - chooser data - school_location_choosers_combined.csv
  
The following code runs the software in estimation mode, inheriting the settings from the simulation setup and using the San Francisco county data setup.  It produces the EDB for all submodels but runs all the model steps identified in the inherited settings file.  

In [2]:
%cd test
!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

C:\projects\development\activitysim_rsg\activitysim\examples\example_estimation\notebooks\test
Configured logging using basicConfig
INFO:activitysim:Configured logging using basicConfig
INFO:activitysim.cli.run:using configs_dir: ['configs_estimation/configs', 'configs']
INFO:activitysim.cli.run:using data_dir: ['data_sf']
INFO:activitysim.cli.run:using output_dir: ['output']
INFO - Read logging configuration from: configs_estimation/configs\logging.yaml
INFO - setting households_sample_size: 0
INFO - setting chunk_size: 0
INFO - setting multiprocess: None
INFO - setting num_processes: None
INFO - setting resume_after: None
INFO - run single process simulation
INFO - open_pipeline
INFO - Set random seed base to 0
INFO - Time to execute open_pipeline : 0.023 seconds (0.0 minutes)
INFO - preload_injectables
INFO - Time to execute preload_injectables : 0.001 seconds (0.0 minutes)
INFO - Reading CSV file data_sf\land_use.csv
INFO - renaming columns: {'ZONE': 'TAZ', 'COUNTY': 'county_id'}
I

DEBUG - school_location: write_omnibus_table: alternatives_combined table_names: ['interaction_sample_alternatives', 'interaction_expression_values']
DEBUG - school_location: write_omnibus_choosers: output\estimation_data_bundle\school_location\school_location_alternatives_combined.csv
INFO - school_location: end estimation
INFO - Initialize Estimator for'workplace_location'
DEBUG - workplace_location: estimate.write_spec: output\estimation_data_bundle\workplace_location\workplace_location_SPEC.csv
DEBUG - workplace_location: write_table write: coefficients
DEBUG - workplace_location: write_table write: size_terms
DEBUG - workplace_location: write_table write: landuse
INFO - Running workplace_location.i1.sample.work_low with 518 persons
INFO - Estimation mode for workplace_location.i1.sample.work_low using unsampled alternatives short_circuit_choices
INFO - Running chunk 1 of 1 size 518
INFO - Running eval_interaction_utilities on 98420 rows
INFO - Estimation mode for workplace_locatio

# Read EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [12]:
from larch import P,X

In [13]:
edb_directory = "output/estimation_data_bundle/school_location/"

def read_csv(filename, **kwargs):
    return pd.read_csv(os.path.join(edb_directory, filename), **kwargs)

### Model Settings

In [14]:
try:
    from yaml import CLoader as yamlLoader
except ImportError:
    from yaml import yamlLoader

with open(os.path.join(edb_directory, "school_location_model_settings.yaml"), 'r') as stream:
    settings = yaml.load(stream, Loader=yamlLoader)


In [15]:
CHOOSER_SEGMENT_COLUMN_NAME = settings['CHOOSER_SEGMENT_COLUMN_NAME']
SEGMENT_IDS = settings['SEGMENT_IDS']
settings

{'SAMPLE_SIZE': 30,
 'SIMULATE_CHOOSER_COLUMNS': ['TAZ', 'school_segment', 'household_id'],
 'CHOOSER_ORIG_COL_NAME': 'TAZ',
 'ALT_DEST_COL_NAME': 'alt_dest',
 'IN_PERIOD': 14,
 'OUT_PERIOD': 8,
 'DEST_CHOICE_COLUMN_NAME': 'school_taz',
 'DEST_CHOICE_LOGSUM_COLUMN_NAME': 'school_taz_logsum',
 'DEST_CHOICE_SAMPLE_TABLE_NAME': 'school_location_sample',
 'SAMPLE_SPEC': 'school_location_sample.csv',
 'SPEC': 'school_location.csv',
 'COEFFICIENTS': 'school_location_coeffs.csv',
 'LOGSUM_SETTINGS': 'tour_mode_choice.yaml',
 'LOGSUM_PREPROCESSOR': 'nontour_preprocessor',
 'LOGSUM_TOUR_PURPOSE': {'university': 'univ',
  'highschool': 'school',
  'gradeschool': 'school'},
 'annotate_persons': {'SPEC': 'annotate_persons_school', 'DF': 'persons'},
 'CHOOSER_TABLE_NAME': 'persons',
 'MODEL_SELECTOR': 'school',
 'CHOOSER_SEGMENT_COLUMN_NAME': 'school_segment',
 'CHOOSER_FILTER_COLUMN_NAME': 'is_student',
 'SEGMENT_IDS': {'university': 3, 'highschool': 2, 'gradeschool': 1},
 'SHADOW_PRICE_TABLE': 's

### Utility specification

In [16]:
spec = read_csv("school_location_SPEC.csv")

In [17]:
spec

,Label,Description,Expression,university,highschool,gradeschool
0,local_dist,NaN,_DIST@skims['DIST'],1,1,1
1,util_dist_0_1,"Distance, piecewise linear from 0 to 1 miles","@_DIST.clip(0,1)",coef_univ_dist_0_1,coef_high_dist_0_1,coef_grade_dist_0_1
2,util_dist_1_2,"Distance, piecewise linear from 1 to 2 miles","@(_DIST-1).clip(0,1)",coef_univ_dist_1_2,coef_high_grade_dist_1_2,coef_high_grade_dist_1_2
3,util_dist_2_5,"Distance, piecewise linear from 2 to 5 miles","@(_DIST-2).clip(0,3)",coef_univ_dist_2_5,coef_high_grade_dist_2_5,coef_high_grade_dist_2_5
4,util_dist_5_15,"Distance, piecewise linear from 5 to 15 miles","@(_DIST-5).clip(0,10)",coef_univ_dist_5_15,coef_high_dist_5_15,coef_grade_dist_5_15
5,util_dist_15_up,"Distance, piecewise linear for 15+ miles",@(_DIST-15.0).clip(0),coef_univ_dist_15_up,coef_high_dist_15_up,coef_grade_dist_15_up
6,util_size_variable,Size variable,@(df['size_term'] * df['shadow_price_size_term...,1,1,1
7,util_utility_adjustment,utility adjustment,@df['shadow_price_utility_adjustment'],1,1,1
8,util_no_attractions,No attractions,@df['size_term']==0,-999,-999,-999
9,util_mode_choice_logsum,Mode choice logsum,mode_choice_logsum,coef_mode_logsum,coef_mode_logsum,coef_mode_logsum


### Remove shadow pricing and pre-existing size expression

In [18]:
spec = spec\
.set_index('Label')\
.drop(index=['util_size_variable', 'util_utility_adjustment'])\
.reset_index()

### Zone size term specification

In [19]:
size_spec = read_csv("school_location_size_terms.csv")

In [20]:
school_size_spec = size_spec \
.query("model_selector == 'school'") \
.drop(columns='model_selector') \
.set_index('segment')
school_size_spec = school_size_spec.loc[:,school_size_spec.max()>0]
school_size_spec

,AGE0519,HSENROLL,COLLFTE,COLLPTE
segment,,,,
university,0.0,0.0,0.592,0.408
gradeschool,1.0,0.0,0.000,0.000
highschool,0.0,1.0,0.000,0.000


### Zone size term coefficients

In [21]:
size_coef = school_size_spec.stack().reset_index()
size_coef.index = size_coef.iloc[:,0] +"_"+ size_coef.iloc[:,1]
size_coef = size_coef.loc[size_coef.iloc[:,2]>0]
size_coef['constrain'] = 'F'
one_each = size_coef.groupby('segment').first().reset_index()
size_coef.loc[one_each.iloc[:,0] +"_"+ one_each.iloc[:,1], 'constrain'] = 'T'
size_coef = size_coef.iloc[:,2:]
size_coef.columns = ['value','constrain']
size_coef.index.name = 'coefficient_name'
size_coef['value'] = np.log(size_coef['value'])
size_coef

,value,constrain
coefficient_name,,
university_COLLFTE,-0.524249,T
university_COLLPTE,-0.896488,F
gradeschool_AGE0519,0.000000,T
highschool_HSENROLL,0.000000,T


### Coefficients

In [22]:
coefficients = read_csv("school_location_coefficients.csv",index_col=0)
coefficients

,value,constrain
coefficient_name,,
coef_univ_dist_0_1,-3.2451,F
coef_univ_dist_1_2,-2.7011,F
coef_univ_dist_2_5,-0.5707,F
coef_univ_dist_5_15,-0.5002,F
coef_univ_dist_15_up,-0.0730,F
coef_high_dist_0_1,-0.9523,F
coef_high_grade_dist_1_2,-0.5700,F
coef_high_grade_dist_2_5,-0.5700,F
coef_high_dist_5_15,-0.1930,F


### Chooser data

In [297]:
x_co = read_csv("school_location_choosers_combined.csv",index_col='person_id')
x_co.head()

,model_choice,override_choice,TAZ,school_segment,household_id
person_id,,,,,
629,13,133,131,3,629
1274,10,188,166,3,1274
27266,10,133,9,3,27266
28012,5,12,10,3,28012
29368,185,13,16,3,29368


### Alternatives Data

In [299]:
x_cv = read_csv("school_location_alternatives_combined.csv", index_col=(0,1))
x_cv.head()

1  \
person_id variable                                                 
629       TAZ                                                1.0   
          mode_choice_logsum                 -1.5183555655982788   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                              2  \
person_id variable                                                
629       TAZ                                               2.0   
          mode_choice_logsum                 -1.353962644303794   
          pick_count                                        1.0   
          prob                                              0.0   
          shadow_price_size_term_adjustment                 1.0   

                                                               3  \
person_id variable                                                 
629       TAZ                                                3.0   
          mode_choice_logsum                 -1.0526039754400893   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               4  \
person_id variable                                                 
629       TAZ                                                4.0   
          mode_choice_logsum                 -1.3907332428502703   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                                 5  \
person_id variable                                                   
629       TAZ                                                  5.0   
          mode_choice_logsum                   -1.2112310544656295   
          pick_count                                           1.0   
          prob                               3.964160745753979e-05   
          shadow_price_size_term_adjustment                    1.0   

                                                               6  \
person_id variable                                                 
629       TAZ                                                6.0   
          mode_choice_logsum                 -0.9024649772476414   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               7  \
person_id variable                                                 
629       TAZ                                                7.0   
          mode_choice_logsum                 -0.8544294523037963   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                               8  \
person_id variable                                                 
629       TAZ                                                8.0   
          mode_choice_logsum                 -0.9528358416884833   
          pick_count                                         1.0   
          prob                                               0.0   
          shadow_price_size_term_adjustment                  1.0   

                                                                 9  \
person_id variable                                                   
629       TAZ                                                  9.0   
          mode_choice_logsum  

### Zone land use data

In [300]:
landuse = read_csv("school_location_landuse.csv", index_col='TAZ')
landuse.head()

,DISTRICT,SD,county_id,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,AGE0519,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
TAZ,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,46,82,20.3,1.0,15.00000,27318,7,...,932.83514,0,0.0,0.00000,0.0,3,5.89564,2.875000,1707.375000,2.870167
2,1,1,1,134,240,31.1,1.0,24.79297,42078,19,...,885.61682,0,0.0,0.00000,0.0,1,5.84871,5.195214,1631.374751,5.178722
3,1,1,1,267,476,14.7,1.0,2.31799,2445,38,...,716.27252,0,0.0,0.00000,0.0,1,5.53231,80.470405,736.891913,72.547987
4,1,1,1,151,253,19.3,1.0,18.00000,22434,20,...,314.00000,0,0.0,0.00000,0.0,2,5.64330,7.947368,1180.736842,7.894233
5,1,1,1,611,1069,52.7,1.0,15.00000,15662,86,...,314.01431,0,0.0,72.14684,0.0,1,5.52555,38.187500,978.875000,36.753679


### Remove choosers with invalid observed choice

In [301]:
SEGMENT_IDS_REVERSE = {v: k for k, v in SEGMENT_IDS.items()}
x_co["school_segment_label"] = x_co["school_segment"].apply(lambda x: SEGMENT_IDS_REVERSE[x])

for segment in school_size_spec.index:
    landuse["total_size_" + segment] = 0
    x_co["total_size_" + segment] = 0 
    for land_use_field in school_size_spec.loc[segment].index:
        landuse["total_size_" + segment] = landuse["total_size_" + segment] + landuse[land_use_field] * school_size_spec.loc[segment][land_use_field]
    x_co["total_size_" + segment] = landuse.loc[x_co["override_choice"]]["total_size_" + segment].tolist()

x_co["total_size_segment"] = 0
for segment in school_size_spec.index:
    labels = "total_size_" + segment
    rows = x_co["school_segment_label"] == segment
    x_co["total_size_segment"][rows] = x_co[labels][rows]
    
x_co["valid_observed_zone"] = x_co["total_size_segment"] > 0
x_co = x_co[x_co["valid_observed_zone"]]

x_co["valid_observed_zone"].value_counts() #True is valid, False is invalid

c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


True    984
Name: valid_observed_zone, dtype: int64

# Data Processing and Estimation Setup

The next step is to transform the EDB for larch for model re-estimation.  

In [302]:
x_ca = larch.util.activitysim.cv_to_ca(x_cv)
x_ca = x_ca[x_ca.index.get_level_values('person_id').isin(x_co.index)]

In [303]:
x_ca_1 = pd.merge(x_ca, landuse, on='TAZ', how='left')
x_ca_1.index = x_ca.index

In [304]:
x_ca_1, x_co = larch.util.activitysim.prevent_overlapping_column_names(x_ca_1, x_co)

### Availability

In [328]:
av = x_ca_1['util_no_attractions'].apply(lambda x: False if x == 1 else True)

In [329]:
d = larch.DataFrames(
    co=x_co,
    ca=x_ca_1,
    av=av
)

### Utility specifications

In [307]:
m = larch.Model()

In [308]:
m.utility_ca = larch.util.activitysim.linear_utility_from_spec(
    spec, x_col='Label', 
    p_col=SEGMENT_IDS, 
    ignore_x=('local_dist',), 
    segment_id=CHOOSER_SEGMENT_COLUMN_NAME,
)
print(m.utility_ca)

  P.coef_univ_dist_0_1 * X('util_dist_0_1*(school_segment==3)')
+ P.coef_univ_dist_1_2 * X('util_dist_1_2*(school_segment==3)')
+ P.coef_univ_dist_2_5 * X('util_dist_2_5*(school_segment==3)')
+ P.coef_univ_dist_5_15 * X('util_dist_5_15*(school_segment==3)')
+ P.coef_univ_dist_15_up * X('util_dist_15_up*(school_segment==3)')
+ P('-999') * X('util_no_attractions*(school_segment==3)')
+ P.coef_mode_logsum * X('util_mode_choice_logsum*(school_segment==3)')
+ P('1') * X('util_sample_of_corrections_factor*(school_segment==3)')
+ P.coef_high_dist_0_1 * X('util_dist_0_1*(school_segment==2)')
+ P.coef_high_grade_dist_1_2 * X('util_dist_1_2*(school_segment==2)')
+ P.coef_high_grade_dist_2_5 * X('util_dist_2_5*(school_segment==2)')
+ P.coef_high_dist_5_15 * X('util_dist_5_15*(school_segment==2)')
+ P.coef_high_dist_15_up * X('util_dist_15_up*(school_segment==2)')
+ P('-999') * X('util_no_attractions*(school_segment==2)')
+ P.coef_mode_logsum * X('util_mode_choice_logsum*(school_segment==2)')
+ P(

In [309]:
m.quantity_ca = sum(
    P(f"{i}_{q}") * X(q) * X(f"{settings['CHOOSER_SEGMENT_COLUMN_NAME']}=={settings['SEGMENT_IDS'][i]}")
    for i in school_size_spec.index
    for q in school_size_spec.columns
    if school_size_spec.loc[i,q]!=0
)

In [310]:
larch.util.activitysim.explicit_value_parameters_from_spec(spec, p_col=SEGMENT_IDS, model=m)

### Coefficients

In [311]:
m.pf

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
-999,-999.0,0.0,0.0,-inf,inf,1,
1,1.0,0.0,0.0,-inf,inf,1,
coef_grade_dist_0_1,0.0,0.0,0.0,-inf,inf,0,
coef_grade_dist_15_up,0.0,0.0,0.0,-inf,inf,0,
coef_grade_dist_5_15,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_0_1,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_15_up,0.0,0.0,0.0,-inf,inf,0,
coef_high_dist_5_15,0.0,0.0,0.0,-inf,inf,0,
coef_high_grade_dist_1_2,0.0,0.0,0.0,-inf,inf,0,
coef_high_grade_dist_2_5,0.0,0.0,0.0,-inf,inf,0,


In [312]:
larch.util.activitysim.apply_coefficients(coefficients, m)
larch.util.activitysim.apply_coefficients(size_coef, m, minimum=-6, maximum=6)

In [313]:
m.pf  # Spot check, confirm coefficients set correctly. 

,value,initvalue,nullvalue,minimum,maximum,holdfast,note
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,
1,1.000000,1.0,1.0,1.0,1.0,1,
coef_grade_dist_0_1,-1.641900,0.0,0.0,NaN,NaN,0,
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,
coef_grade_dist_5_15,-0.203100,0.0,0.0,NaN,NaN,0,
coef_high_dist_0_1,-0.952300,0.0,0.0,NaN,NaN,0,
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,
coef_high_dist_5_15,-0.193000,0.0,0.0,NaN,NaN,0,
coef_high_grade_dist_1_2,-0.570000,0.0,0.0,NaN,NaN,0,
coef_high_grade_dist_2_5,-0.570000,0.0,0.0,NaN,NaN,0,


In [314]:
m.dataservice = d

### Survey choice

In [315]:
m.choice_co_code = 'override_choice'

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has two built-in estimation methods: BHHH and SLSQP.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.  SLSQP is safer, but slower, and may need additional iterations.

In [317]:
# m.estimate(method='SLSQP', options={'maxiter':1000})
m.estimate(method='BHHH', options={'maxiter':1000})

req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,std_err,t_stat,robust_std_err,robust_t_stat,unconstrained_std_err,unconstrained_t_stat,constrained,best
-999,-999.000000,-999.0,-999.0,-999.0,-999.0,1,,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,fixed value,-999.000000
1,1.000000,1.0,1.0,1.0,1.0,1,,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,fixed value,1.000000
coef_grade_dist_0_1,-3.727781,0.0,0.0,NaN,NaN,0,,2.315438e-01,-1.609968e+01,2.210235e-01,-1.686600e+01,2.315438e-01,-1.609968e+01,,-3.727781
coef_grade_dist_15_up,-0.046000,0.0,0.0,NaN,NaN,0,,1.717021e-15,-2.679059e+13,1.869788e-15,-2.460172e+13,1.717021e-15,-2.679059e+13,,-0.046000
coef_grade_dist_5_15,-0.421906,0.0,0.0,NaN,NaN,0,,1.219025e-01,-3.461011e+00,1.258530e-01,-3.352371e+00,1.219025e-01,-3.461011e+00,,-0.421906
coef_high_dist_0_1,-1.981339,0.0,0.0,NaN,NaN,0,,6.501108e-01,-3.047694e+00,7.103136e-01,-2.789386e+00,6.501108e-01,-3.047694e+00,,-1.981339
coef_high_dist_15_up,-0.188200,0.0,0.0,NaN,NaN,0,,1.056981e-15,-1.780543e+14,1.150513e-15,-1.635791e+14,1.056981e-15,-1.780543e+14,,-0.188200
coef_high_dist_5_15,-0.257137,0.0,0.0,NaN,NaN,0,,2.088883e-01,-1.230979e+00,1.901204e-01,-1.352497e+00,2.088883e-01,-1.230979e+00,,-0.257137
coef_high_grade_dist_1_2,-1.189074,0.0,0.0,NaN,NaN,0,,1.577002e-01,-7.540092e+00,1.617762e-01,-7.350116e+00,1.577002e-01,-7.540092e+00,,-1.189074
coef_high_grade_dist_2_5,-1.098452,0.0,0.0,NaN,NaN,0,,6.475680e-02,-1.696272e+01,6.873185e-02,-1.598170e+01,6.475680e-02,-1.696272e+01,,-1.098452


c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 2.669099071048399e-15 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")
c:\programdata\anaconda3\envs\asimtest2\lib\site-packages\larch\linalg\__init__.py:18: UserWarning: minimum eig 9.716975768626752e-16 in general_inverse
  warnings.warn(f"minimum eig {min_eig} in general_inverse")


┣          loglike: -3097.6915321141814
┣                x: -999                       -999.000000
┃                   1                             1.000000
┃                   coef_grade_dist_0_1          -3.727781
┃                   coef_grade_dist_15_up        -0.046000
┃                   coef_grade_dist_5_15         -0.421906
┃                   coef_high_dist_0_1           -1.981339
┃                   coef_high_dist_15_up         -0.188200
┃                   coef_high_dist_5_15          -0.257137
┃                   coef_high_grade_dist_1_2     -1.189074
┃                   coef_high_grade_dist_2_5     -1.098452
┃                   coef_mode_logsum              0.484746
┃                   coef_univ_dist_0_1           -7.518143
┃                   coef_univ_dist_15_up         -0.073000
┃                   coef_univ_dist_1_2           -4.426815
┃                   coef_univ_dist_2_5           -1.169154
┃                   coef_univ_dist_5_15          -1.537447
┃                   gradeschool_AGE0519           0.000000
┃                   highschool_HSENROLL           0.000000
┃                   university_COLLFTE           -0.524249
┃                   university_COLLPTE            1.807565
┃                   dtype: float64
┣        tolerance: 3.973751662391261e-08
┣            steps: array([], dtype=float64)
┣          message: 'Optimization terminated successfully.'
┣     elapsed_time: datetime.timedelta(microseconds=123003)
┣           method: 'BHHH'
┣          n_cases: 984
┣ iteration_number: 0
┣          logloss: 3.148060500116038

### Estimated coefficients

In [318]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
-999,-999.,NA,NA,,-999.00,fixed value
1,1.00,NA,NA,,1.00,fixed value
coef_grade_dist_0_1,-3.73,0.232,-16.10,***,0.00,
coef_grade_dist_15_up,-0.0460,2.35e-15,-19579306130383.17,***,0.00,
coef_grade_dist_5_15,-0.422,0.122,-3.46,***,0.00,
coef_high_dist_0_1,-1.98,0.650,-3.05,**,0.00,
coef_high_dist_15_up,-0.188,4.48e-16,-420173649990679.69,***,0.00,
coef_high_dist_5_15,-0.257,0.209,-1.23,,0.00,
coef_high_grade_dist_1_2,-1.19,0.158,-7.54,***,0.00,
coef_high_grade_dist_2_5,-1.10,0.0648,-16.96,***,0.00,


# Output Estimation Results

In [319]:
est_names = [j for j in coefficients.index if j in m.pf.index]
coefficients.loc[est_names, 'value'] = m.pf.loc[est_names, 'value']

In [320]:
os.makedirs(os.path.join(edb_directory,'estimated'), exist_ok=True)

### Write the re-estimated coefficients file

In [321]:
coefficients.reset_index().to_csv(
    os.path.join(
        edb_directory, 
        'estimated',
        "school_location_coefficients_revised.csv",
    ),
    index=False,
)

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [323]:
m.to_xlsx(
    os.path.join(
        edb_directory, 
        'estimated',
        "school_location_model_estimation.xlsx"
    ), data_statistics=False
)

### Write updated size coefficients

In [324]:
# Write size coefficients into size_spec
for c in school_size_spec.columns:
    for i in school_size_spec.index:
        param_name = f"{i}_{c}"
        j = (size_spec['segment'] == i) & (size_spec['model_selector'] == 'school')
        try:
            size_spec.loc[j,c] = np.exp(m.get_value(param_name))
        except KeyError:
            pass


In [325]:
# Rescale each row to total 1, not mathematically needed
# but to maintain a consistent approach from existing ASim
size_spec.iloc[:,2:] = (size_spec.iloc[:,2:].div(size_spec.iloc[:,2:].sum(1), axis=0))

In [326]:
size_spec.to_csv(
    os.path.join(edb_directory,'estimated',"school_location_size_terms.csv"), 
    index=False,
)

# Next Steps

The final step is to either manually or automatically copy the `school_location_coefficients_revised.csv` file and `school_location_size_terms.csv` file to the configs folder, rename them to `school_location_coeffs.csv` and `destination_choice_size_terms.csv`, and run ActivitySim in simulation mode.

In [327]:
pd.read_csv(os.path.join(edb_directory,'estimated',"school_location_coefficients_revised.csv"))

,coefficient_name,value,constrain
0,coef_univ_dist_0_1,-7.518143,F
1,coef_univ_dist_1_2,-4.426815,F
2,coef_univ_dist_2_5,-1.169154,F
3,coef_univ_dist_5_15,-1.537447,F
4,coef_univ_dist_15_up,-0.073000,F
5,coef_high_dist_0_1,-1.981339,F
6,coef_high_grade_dist_1_2,-1.189074,F
7,coef_high_grade_dist_2_5,-1.098452,F
8,coef_high_dist_5_15,-0.257137,F
9,coef_high_dist_15_up,-0.188200,F


In [52]:
pd.read_csv(os.path.join(edb_directory,'estimated',"school_location_size_terms.csv"))

,segment,model_selector,TOTHH,RETEMPN,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,AGE0519,HSENROLL,COLLFTE,COLLPTE
0,work_low,workplace,0.000000,0.129129,0.193193,0.383383,0.120120,0.010010,0.164164,0.000,0.000000,0.000000,0.000000
1,work_med,workplace,0.000000,0.120120,0.197197,0.325325,0.139139,0.008008,0.210210,0.000,0.000000,0.000000,0.000000
2,work_high,workplace,0.000000,0.110000,0.207000,0.284000,0.154000,0.006000,0.239000,0.000,0.000000,0.000000,0.000000
3,work_veryhigh,workplace,0.000000,0.093000,0.270000,0.241000,0.146000,0.004000,0.246000,0.000,0.000000,0.000000,0.000000
4,university,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.088566,0.911434
5,gradeschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000,0.000000,0.000000,0.000000
6,highschool,school,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,1.000000,0.000000,0.000000
7,escort,non_mandatory,0.000000,0.225000,0.000000,0.144000,0.000000,0.000000,0.000000,0.465,0.166000,0.000000,0.000000
8,shopping,non_mandatory,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,eatout,non_mandatory,0.000000,0.742000,0.000000,0.258000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
